In [1]:
import logging
import os
import sys
from datetime import datetime

import pandas as pd
import yfinance as yf
from pandas_datareader import data as pdr
from pyrate_limiter import Duration, Limiter, RequestRate
from requests import Session
from requests_cache import CacheMixin, SQLiteCache
from requests_ratelimiter import LimiterMixin, MemoryQueueBucket

yf.pdr_override()

In [2]:
notebooks_dir_parent = os.path.relpath(os.path.dirname(os.getcwd()))
sm_dir = os.path.join(notebooks_dir_parent, "stock-market-analysis", "src")

sys.path.append(sm_dir)

In [3]:
class CachedLimiterSession(CacheMixin, LimiterMixin, Session):
    def __init__(self, timeout=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.timeout = timeout

    def request(self, *args, **kwargs):
        # Pass the stored timeout to the request method
        kwargs["timeout"] = self.timeout
        return super().request(*args, **kwargs)


session = CachedLimiterSession(
    limiter=Limiter(RequestRate(2, Duration.SECOND * 5)),  # max 2 requests per 5 seconds
    bucket_class=MemoryQueueBucket,
    backend=SQLiteCache("yfinance.cache"),
    timeout=5,
)


def get_logger(name):
    # Create a logger
    logger = logging.getLogger(name)

    # Set the logging level (adjust as needed)
    logger.setLevel(logging.DEBUG)

    # Create a console handler and set the level
    ch = logging.StreamHandler()
    ch.setLevel(logging.DEBUG)

    # Create a formatter and add it to the handler
    formatter = logging.Formatter("%(asctime)s | %(levelname)s:%(name)s - %(message)s")
    ch.setFormatter(formatter)

    # Add the handler to the logger
    logger.addHandler(ch)
    return logger


logger = get_logger("notebook")

In [44]:
data = pdr.get_data_yahoo(
    "ADANIENT.NS",start="1600-01-01",end=datetime.today(), interval="1d"
).reset_index()
data

[*********************100%%**********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2002-07-01,1.364148,1.401700,1.334259,1.362615,-0.010452,1080397
1,2002-07-02,1.386372,1.390971,1.356484,1.373344,-0.010535,1016147
2,2002-07-03,1.371811,1.387139,1.364914,1.373344,-0.010535,980394
3,2002-07-04,1.379475,1.401700,1.379475,1.381008,-0.010593,972747
4,2002-07-05,1.388672,1.388672,1.372578,1.377942,-0.010570,974496
...,...,...,...,...,...,...,...
5359,2024-01-24,2901.000000,2929.100098,2871.250000,2903.449951,2903.449951,5807940
5360,2024-01-25,2903.449951,2926.350098,2804.399902,2893.600098,2893.600098,2601969
5361,2024-01-29,2929.000000,3092.699951,2923.050049,3064.649902,3064.649902,4402976
5362,2024-01-30,3081.000000,3129.899902,3020.899902,3091.100098,3091.100098,3115506


In [33]:
from yf import get_ticker_data, clean_ticker_data, resample, basic_preprocess

df = get_ticker_data(
    "AAPL",
    start_date=datetime.strptime("2010-01-01", "%Y-%m-%d"),
    end_date=datetime.strptime("2015-01-01", "%Y-%m-%d"),
)
df = clean_ticker_data(df)
df = basic_preprocess(resample(df))
df

[*********************100%%**********************]  1 of 1 completed


,datetime,open,high,low,close,adj_close,volume
0,2010-01-04,7.622500,7.660714,7.585000,7.643214,6.478997,493729600.0
1,2010-01-05,7.664286,7.699643,7.616071,7.656429,6.490199,601904800.0
2,2010-01-06,7.656429,7.686786,7.526786,7.534643,6.386964,552160000.0
3,2010-01-07,7.562500,7.571429,7.466071,7.520714,6.375158,477131200.0
4,2010-01-08,7.510714,7.571429,7.466429,7.570714,6.417541,447610800.0
...,...,...,...,...,...,...,...
1818,2014-12-27,28.165833,28.650832,28.143333,28.490833,25.503325,126721200.0
1819,2014-12-28,28.306667,28.671666,28.284166,28.484167,25.497358,118558400.0
1820,2014-12-29,28.447500,28.692499,28.424999,28.477501,25.491390,110395600.0
1821,2014-12-30,28.410000,28.480000,28.027500,28.129999,25.180319,119526000.0


In [48]:
start_date=datetime.strptime("2018-01-01", "%Y-%m-%d")
end_date=datetime.strptime("2015-01-01", "%Y-%m-%d")

# d = end_date-end_date
# d.days
end_date.date()

datetime.date(2015, 1, 1)

In [22]:
from models import TickerDataModel
TickerDataModel(data=df.to_dict(orient="records")).model_dump()["data"]

[{'datetime': Timestamp('2002-07-01 00:00:00'),
  'open': 1.3641480207443237,
  'high': 1.4017000198364258,
  'low': 1.334259033203125,
  'close': 1.3626149892807007,
  'adj_close': -0.010452346876263618,
  'volume': 1080397.0},
 {'datetime': Timestamp('2002-07-02 00:00:00'),
  'open': 1.3863719701766968,
  'high': 1.3909709453582764,
  'low': 1.356484055519104,
  'close': 1.3733439445495605,
  'adj_close': -0.010534646920859814,
  'volume': 1016147.0},
 {'datetime': Timestamp('2002-07-03 00:00:00'),
  'open': 1.371811032295227,
  'high': 1.3871389627456665,
  'low': 1.3649139404296875,
  'close': 1.3733439445495605,
  'adj_close': -0.010534646920859814,
  'volume': 980394.0},
 {'datetime': Timestamp('2002-07-04 00:00:00'),
  'open': 1.3794749975204468,
  'high': 1.4017000198364258,
  'low': 1.3794749975204468,
  'close': 1.3810080289840698,
  'adj_close': -0.010593434795737267,
  'volume': 972747.0},
 {'datetime': Timestamp('2002-07-05 00:00:00'),
  'open': 1.3886719942092896,
  'high

In [19]:
from models import TickerSummaryModel

def get_ticker_info(ticker_code: str):
    # Create a Ticker object using yfinance with the provided ticker code and session
    ticker_info = yf.Ticker(ticker_code, session=session)

    # Retrieve information about the ticker
    ticker_info = ticker_info.info
    if "symbol" in ticker_info:
        ticker_info = TickerSummaryModel(
            ticker_code = ticker_info["symbol"],
            name = ticker_info["shortName"],
            exchange = ticker_info["exchange"]
        )
        return ticker_info.model_dump()
    else:
        logger.error("Invalid Ticker code '%s'", ticker_code)
        # raise ValueError("Invalid Ticker code '%s'", ticker_code)


get_ticker_info("AAPL")

{'name': 'Apple Inc.', 'ticker_code': 'AAPL', 'exchange': 'NMS'}

In [6]:
datetime.strptime("2015-01-01", "%Y-%m-%d")

datetime.datetime(2015, 1, 1, 0, 0)

In [33]:
ticker_list = pd.read_csv("us_symbols.csv", keep_default_na=False)
ticker_list.rename(columns={"ticker": "ticker_code"}, inplace=True)
ticker_list

,ticker_code,name,exchange
0,A,Agilent Technologies Inc.,NYSE
1,AA,Alcoa Corporation,NYSE
2,AACG,ATA Creativity Global,NASDAQ
3,AACI,Armada Acquisition Corp. I,NASDAQ
4,AACT,Ares Acquisition Corporation II,NYSE
...,...,...,...
5674,ZVRA,Zevra Therapeutics Inc.,NASDAQ
5675,ZVSA,ZyVersa Therapeutics Inc.,NASDAQ
5676,ZWS,Zurn Elkay Water Solutions Corporation,NYSE
5677,ZYME,Zymeworks Inc.,NASDAQ


In [34]:
from models import TickerBaseListModel, TickerSummaryModel
from db import tickers_info_collection

ticker_list["created_at"] = datetime.utcnow()
ticker_list = TickerBaseListModel(ticker_list.to_dict(orient="records")).model_dump()
# tickers_info_collection.insert_many(ticker_list)
# ticker_list

In [35]:
from db import PyObjectId
from models import TickerSummaryModel
TickerSummaryModel(**tickers_info_collection.find_one({"_id": PyObjectId("65b9f531e4178f1bf53b7f4c")},{"_id": 0}))

TickerSummaryModel(name='Agilent Technologies Inc.', ticker_code='A', exchange='NYSE')

In [9]:
from manager import PortfolioManager
from models import PortfolioModel

pm = PortfolioManager(username="john_doe")

# pm.create_portfolio(
#     PortfolioModel(
#         username=pm.username,
#         portfolio_name="portfolio1",
#         # tickers: Optional[List[TickerSummaryModel]] = None
#         created_at=datetime.utcnow()
#     )
# )

# pm.create_portfolio(
#     PortfolioModel(
#         username=pm.username,
#         portfolio_name="portfolio2",
#         # tickers: Optional[List[TickerSummaryModel]] = None
#         created_at=datetime.utcnow()
#     )
# )

pm.get_portfolios().model_dump()

[{'portfolio_name': 'portfolio1',
  'tickers': None,
  'created_at': datetime.datetime(2024, 1, 31, 8, 53, 51, 514000),
  'updated_at': None},
 {'portfolio_name': 'portfolio2',
  'tickers': None,
  'created_at': datetime.datetime(2024, 1, 31, 8, 54, 28, 279000),
  'updated_at': None}]

In [21]:
x = PortfolioModel(
        username=pm.username,
        portfolio_name="portfolio2",
        # tickers: Optional[List[TickerSummaryModel]] = None
        created_at=datetime.utcnow()
    )
x

PortfolioModel(username='john_doe', portfolio_name='portfolio2', tickers=None, created_at=datetime.datetime(2024, 1, 31, 11, 12, 41, 849866), updated_at=None)

In [24]:
from models import PortfolioPreviewModel
PortfolioPreviewModel(**x.model_dump())

PortfolioPreviewModel(portfolio_name='portfolio2', tickers=None, created_at=datetime.datetime(2024, 1, 31, 11, 12, 41, 849866), updated_at=None)

In [43]:
from db import ticker_db

ticker_data_collection = ticker_db["GOOG"]
data = ticker_data_collection.find({"datetime": {"$gte": None, "$lte": datetime.today()}},{"_id":0})

pd.DataFrame(list(data)).empty

True